# Neural Network for single lipids

### Imports

In [13]:
# Basics
import os
import random
import numpy as np
import pandas as pd
# Tensorflow
import tensorflow as tf
import tensorflow_addons as tfa
# Keras
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Activation
from keras.initializers import he_normal
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
# Sklearn
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

### Reproducibility

In [14]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Data loading

In [15]:
# Dataset path 
dataset_dir = 'data/processed_data'
# Training path
training_input_path = os.path.join(dataset_dir, 'train_features.parquet')
training_output_path = os.path.join(dataset_dir, 'train_targets.parquet')
# Test path
testing_input_path = os.path.join(dataset_dir, 'test_features.parquet')
testing_output_path = os.path.join(dataset_dir, 'test_targets.parquet')

In [16]:
# Load dataset into Pandas dataframes
training_input = pd.read_parquet(training_input_path)
training_output = pd.read_parquet(training_output_path)

testing_input = pd.read_parquet(testing_input_path)
testing_output = pd.read_parquet(testing_output_path)

# Number of output nodes (lipids) for the model
OUTPUT_NODES = training_output.shape[1]
# Number of input nodes (genes) for the model
input_dim = training_input.shape[1]
# Batch size
batch_size = 64
# Remove Nans from lipids
remove_nans_lipids = False

### Definition of the model

In [28]:
def lr_schedule(epoch):
    """
    Set the learning rate considering the epoch's number
    :param epoch: epoch's number
    :return: learning rate
    """
    initial_learning_rate = 0.1  # Set initial learning rate
    decay_factor = 0.9  # Set decay factor
    lr = initial_learning_rate * decay_factor ** (epoch - 1)  # Compute learnign rate
    return lr

# Create a Sequential model
def build_model(summary=False):
    """
    Build the neural network
    :param summary: if True, print the summary of the model, if False, do not print
    :return: the model
    """
    model = Sequential()

    # Add the input layer with input_dim input nodes
    model.add(Dense(64, input_dim=input_dim, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(32, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(16, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(8, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(4, activation='gelu', kernel_initializer=he_normal(seed=seed)))

    # Add the output layer with 1 node (for single regression)
    model.add(Dense(1, activation='linear'))

    if summary:
        # Display the model summary
        model.summary()

    return model

# Print the model summary
build_model(True)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_68 (Dense)            (None, 64)                32064     
                                                                 
 dense_69 (Dense)            (None, 32)                2080      
                                                                 
 dense_70 (Dense)            (None, 16)                528       
                                                                 
 dense_71 (Dense)            (None, 8)                 136       
                                                                 
 dense_72 (Dense)            (None, 4)                 36        
                                                                 
 dense_73 (Dense)            (None, 1)                 5         
                                                                 
Total params: 34,849
Trainable params: 34,849
Non-tra

In [29]:
# Use early stopping on the validation
early_stopping = EarlyStopping(monitor='val_loss',  # Metric chose is validation loss (MSE)
                               patience=10,         # Number of epochs with no improvement after which training stops
                               restore_best_weights=True)  # Restores model weights from the epoch with the best value of the monitored metric

In [30]:
def data_generator(X, y, batch_size):
    """
    Data augmentation
    :param X: input DataFrame (genes)
    :param y: output DataFrame (single lipid)
    :param batch_size: size of the mini batch
    :return batch_X: dataset containing random modification on the input DataFrame with size corresponding to batch size
    :return batch_y: dataset containing output with size corresponding to batch size
    """
    # Set noise std and scale factor for random modifications
    noise_std = 0.2
    scale_factor_range = 0.1
    while True:
        indices = np.random.choice(X.shape[0], batch_size, replace=False)
        batch_df = X.iloc[indices]

        # Create a copy of the batch for augmentation
        augmented_batch_df = batch_df.copy()

        # Random scaling
        scale_factor = np.random.uniform(1 - scale_factor_range, 1 + scale_factor_range)
        augmented_batch_df *= scale_factor

        # Add Gaussian noise to all features
        augmented_batch_df += np.random.normal(loc=0, scale=noise_std, size=augmented_batch_df.shape)

        batch_X = augmented_batch_df.values
        batch_y = y.iloc[indices].values

        yield batch_X, batch_y

### Training

In [ ]:
# DataFrame for results
lipid_names = list(map(lambda s: s.strip(), training_output.columns.values))
lipids_metrics_avg = pd.DataFrame(columns=['Loss', 'R2'], index=lipid_names)

# Train a model for each lipid
for j in range(OUTPUT_NODES):
    print('#'*72)
    print(f'Start training for lipid {lipid_names[j]}')
    print(np.std(training_output.iloc[:, j]))

    # Define the K-fold Cross Validator
    num_folds = 5
    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

    # Metrics for each fold
    loss_per_fold = np.zeros((num_folds,))
    r2_per_fold = np.zeros((num_folds,))

    # K-fold Cross Validation model evaluation
    fold_no = 1

    # K-fold Cross Validation model evaluation
    for input_indices, output_indices in k_fold.split(training_input, training_output):
        # Build the model
        model = build_model()

        # Compile the model
        model.compile(optimizer='adam', loss='mean_squared_error',
                      metrics=[metrics.mean_squared_error, metrics.mean_absolute_error, tfa.metrics.RSquare(), metrics.mean_absolute_percentage_error])

        print('------------------------------------------------------------------------')
        print(f'Training for fold {fold_no} ...')

        scaler = MinMaxScaler()
        scaler.fit(training_output.iloc[input_indices])
        training_output_scaled = pd.DataFrame(scaler.transform(training_output.iloc[input_indices]), index=training_output.iloc[input_indices].index, columns=training_output.iloc[input_indices].columns)
        validation_output_scaled = pd.DataFrame(scaler.transform(training_output.iloc[output_indices]), index=training_output.iloc[output_indices].index, columns=training_output.iloc[output_indices].columns)

        train_generator = data_generator(training_input.iloc[input_indices], training_output_scaled.iloc[:, j], batch_size)
        # Train the model
        history = model.fit(
            train_generator,
            epochs=50,  # Adjust the number of epochs as needed
            steps_per_epoch=len(input_indices) // batch_size,
            validation_data=(training_input.iloc[output_indices], validation_output_scaled.iloc[:, j]),
            callbacks=[early_stopping]
        ).history

        # Generate generalization metrics
        scores = model.evaluate(training_input.iloc[output_indices], validation_output_scaled.iloc[:, j], verbose=0)
        print(scores)
        loss_per_fold[fold_no-1] = scores[0]
        r2_per_fold[fold_no-1] = scores[3]

        # Increase fold number
        fold_no = fold_no + 1

    # Computing the values
    mean_loss = loss_per_fold.mean()
    mean_r2 = r2_per_fold.mean()
    lipids_metrics_avg.loc[lipid_names[0]] = [mean_loss, mean_r2]
    print('#'*72)
    print(f'Finish training for lipid {lipid_names[0]}')
    print(f'Mean loss: {mean_loss}')
    print(f'Mean r2: {mean_r2}')

# Saving the values
lipids_metrics_avg.to_csv('lipids_metrics_avg_neural_network_single_lipid.csv')

########################################################################
Start training for lipid LPC O- 18:3
0.00020291377902284177
------------------------------------------------------------------------
Training for fold 1 ...


/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/pyth

Epoch 1/50
782/782 [==============================] - 12s 12ms/step - loss: 0.0105 - mean_squared_error: 0.0105 - mean_absolute_error: 0.0788 - r_square: 0.0071 - mean_absolute_percentage_error: 1186476.6250 - val_loss: 0.0093 - val_mean_squared_error: 0.0093 - val_mean_absolute_error: 0.0751 - val_r_square: 0.1374 - val_mean_absolute_percentage_error: 1386369.8750
Epoch 2/50
782/782 [==============================] - 10s 12ms/step - loss: 0.0091 - mean_squared_error: 0.0091 - mean_absolute_error: 0.0738 - r_square: 0.1222 - mean_absolute_percentage_error: 1112427.6250 - val_loss: 0.0090 - val_mean_squared_error: 0.0090 - val_mean_absolute_error: 0.0733 - val_r_square: 0.1685 - val_mean_absolute_percentage_error: 1237914.8750
Epoch 3/50
782/782 [==============================] - 8s 10ms/step - loss: 0.0091 - mean_squared_error: 0.0091 - mean_absolute_error: 0.0738 - r_square: 0.1419 - mean_absolute_percentage_error: 1036802.6875 - val_loss: 0.0090 - val_mean_squared_error: 0.0090 - val

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/pyth

Epoch 1/50
782/782 [==============================] - 8s 8ms/step - loss: 0.0116 - mean_squared_error: 0.0116 - mean_absolute_error: 0.0833 - r_square: 0.0872 - mean_absolute_percentage_error: 1430679.3750 - val_loss: 0.0118 - val_mean_squared_error: 0.0118 - val_mean_absolute_error: 0.0862 - val_r_square: 0.0689 - val_mean_absolute_percentage_error: 960322.5625
Epoch 2/50
782/782 [==============================] - 9s 11ms/step - loss: 0.0108 - mean_squared_error: 0.0108 - mean_absolute_error: 0.0808 - r_square: 0.1429 - mean_absolute_percentage_error: 1262804.6250 - val_loss: 0.0105 - val_mean_squared_error: 0.0105 - val_mean_absolute_error: 0.0793 - val_r_square: 0.1713 - val_mean_absolute_percentage_error: 856483.4375
Epoch 3/50
782/782 [==============================] - 8s 11ms/step - loss: 0.0106 - mean_squared_error: 0.0106 - mean_absolute_error: 0.0800 - r_square: 0.1672 - mean_absolute_percentage_error: 1331328.3750 - val_loss: 0.0104 - val_mean_squared_error: 0.0104 - val_mean

...

Epoch 1/50
782/782 [==============================] - 12s 9ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - mean_absolute_error: 0.0193 - r_square: 0.0183 - mean_absolute_percentage_error: 4086865.5000 - val_loss: 0.0035 - val_mean_squared_error: 0.0035 - val_mean_absolute_error: 0.0187 - val_r_square: 0.0759 - val_mean_absolute_percentage_error: 4533773.0000
Epoch 2/50
782/782 [==============================] - 6s 8ms/step - loss: 0.0028 - mean_squared_error: 0.0028 - mean_absolute_error: 0.0164 - r_square: 0.1280 - mean_absolute_percentage_error: 3571442.2500 - val_loss: 0.0033 - val_mean_squared_error: 0.0033 - val_mean_absolute_error: 0.0151 - val_r_square: 0.1161 - val_mean_absolute_percentage_error: 3281248.7500
Epoch 3/50
782/782 [==============================] - 7s 9ms/step - loss: 0.0031 - mean_squared_error: 0.0031 - mean_absolute_error: 0.0173 - r_square: 0.1360 - mean_absolute_percentage_error: 3619299.7500 - val_loss: 0.0031 - val_mean_squared_error: 0.0031 - val_mea

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/pyth

Epoch 1/50
782/782 [==============================] - 9s 9ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - mean_absolute_error: 0.0222 - r_square: -0.0540 - mean_absolute_percentage_error: 4361674.0000 - val_loss: 0.0028 - val_mean_squared_error: 0.0028 - val_mean_absolute_error: 0.0158 - val_r_square: 0.1683 - val_mean_absolute_percentage_error: 2048876.5000
Epoch 2/50
782/782 [==============================] - 12s 15ms/step - loss: 0.0032 - mean_squared_error: 0.0032 - mean_absolute_error: 0.0186 - r_square: 0.1311 - mean_absolute_percentage_error: 3581200.2500 - val_loss: 0.0027 - val_mean_squared_error: 0.0027 - val_mean_absolute_error: 0.0155 - val_r_square: 0.1968 - val_mean_absolute_percentage_error: 1939021.8750
Epoch 3/50
782/782 [==============================] - 11s 14ms/step - loss: 0.0033 - mean_squared_error: 0.0033 - mean_absolute_error: 0.0179 - r_square: 0.1352 - mean_absolute_percentage_error: 3726073.5000 - val_loss: 0.0028 - val_mean_squared_error: 0.0028 - val

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/pyth

Epoch 1/50
782/782 [==============================] - 7s 8ms/step - loss: 0.0035 - mean_squared_error: 0.0035 - mean_absolute_error: 0.0189 - r_square: -0.0132 - mean_absolute_percentage_error: 4027471.2500 - val_loss: 0.0035 - val_mean_squared_error: 0.0035 - val_mean_absolute_error: 0.0152 - val_r_square: 0.0773 - val_mean_absolute_percentage_error: 3138970.2500
Epoch 2/50
782/782 [==============================] - 6s 8ms/step - loss: 0.0030 - mean_squared_error: 0.0030 - mean_absolute_error: 0.0167 - r_square: 0.1342 - mean_absolute_percentage_error: 3686595.7500 - val_loss: 0.0034 - val_mean_squared_error: 0.0034 - val_mean_absolute_error: 0.0153 - val_r_square: 0.1202 - val_mean_absolute_percentage_error: 3290879.2500
Epoch 3/50
782/782 [==============================] - 5s 6ms/step - loss: 0.0029 - mean_squared_error: 0.0029 - mean_absolute_error: 0.0162 - r_square: 0.1661 - mean_absolute_percentage_error: 3505039.7500 - val_loss: 0.0035 - val_mean_squared_error: 0.0035 - val_mea

### Output processing

In [ ]:
# Print k best and worst lipids for average between folds of metrics loss and r2
k = 5

def format_names(df: pd.DataFrame, just=15):
    return ', '.join([name.rjust(just) for name in df.index.values])

def format_values(df: pd.DataFrame, col: str, just=15):
    return ', '.join([f'{val:.5e}'.rjust(just) for val in df[col].values])

best_losses = lipids_metrics_avg.nsmallest(k, 'Loss')
worst_losses = lipids_metrics_avg.nlargest(k, 'Loss')
print("Loss:")
print("  Best:")
print(f"  {format_names(best_losses)}")
print(f"  {format_values(best_losses, 'Loss')}")
print("  Worst:")
print(f"  {format_names(worst_losses)}")
print(f"  {format_values(worst_losses, 'Loss')}")

best_r2s = lipids_metrics_avg.nlargest(k, 'R2')
worst_r2s = lipids_metrics_avg.nsmallest(k, 'R2')
print("R2:")
print("  Best:")
print(f"  {format_names(best_r2s)}")
print(f"  {format_values(best_r2s, 'R2')}")
print("  Worst:")
print(f"  {format_names(worst_r2s)}")
print(f"  {format_values(worst_r2s, 'R2')}")